# Automated Prompt Engineering with DSPy

This notebook shows how we can pull data from the TensorZero data model in ClickHouse and use it to optimize a prompt for a function using DSPy.
Given that there are many strategies for prompt optimization in DSPy, we can use the same code skeleton to try a lot of different strategies.
At a high level the notebook below does the following:

1. Read the TensorZero configuration file and convert the function schemas of interest into a DSPy signature.
2. Pull data from ClickHouse and convert it into a DSPy dataset.
3. Run a prompt optimization loop using one of the teleprompting classes supported by DSPy.
4. Parse the optimized prompt from the history and write it to a minijinja file.

**Note:** DSPy does not model the chat completion interface commonly used by language models. So, we only support functions that have inputs into the user prompt, that only use text output, that are single-turn functions, and that have a flat JSON schema for input, i.e. functions that take a list of primitive types as input into the user schema and output text or a flat JSON object.

To get started:

- Set the `CLICKHOUSE_NATIVE_URL` environment variable. It should correspond to the native protocol port (i.e. not the HTTP port) of your ClickHouse instance. This is typically port 9000.
- Set the `OPENAI_API_KEY` environment variable.
- Update the following parameters to those that apply to your use case.


In [ ]:
CONFIG_PATH = "../../examples/haiku_hidden_preferences/config/tensorzero.toml"

FUNCTION_NAME = "write_haiku"
BASE_VARIANT_NAME = "initial_prompt_gpt4o_mini"
DEV_FRACTION = 0.1
TEST_FRACTION = 0.1


METRIC_NAME = "haiku_score"
MAX_SAMPLES = 1000
FLOAT_METRIC_THRESHOLD = 0.5

In [ ]:
import json
import logging
import os
from functools import partial
from pathlib import Path
from typing import Dict, Optional
from urllib.parse import urljoin

import dsp
import dspy
import numpy as np
import pandas as pd
import toml
from clickhouse_driver import Client
from dspy.datasets.dataset import Dataset

In [ ]:
logger = logging.getLogger(__name__)

You can swap the client below for any of the ones supported [here](https://dspy-docs.vercel.app/api/category/language-model-api-clients) in case you want DSPy to use a different language model.


In [ ]:
model_name = "gpt-4o-mini-2024-07-18"
lm_client = dspy.OpenAI(model=model_name)
dspy.configure(lm=lm_client)

Load the TensorZero configuration file.


In [ ]:
config_path = Path(CONFIG_PATH)

assert config_path.exists(), f"{CONFIG_PATH} does not exist"
assert config_path.is_file(), f"{CONFIG_PATH} is not a file"

with config_path.open("r") as f:
    config = toml.load(f)

Retrieve the function configuration for the function we are optimizing.


In [ ]:
function_config = config["functions"][FUNCTION_NAME]

In [ ]:
def is_flat_schema(schema: dict):
    """Check if a JSON schema (given as a dict) is flat."""
    if not isinstance(schema, dict):
        return False

    if "type" not in schema or schema["type"] != "object":
        return False

    if "properties" not in schema:
        return True

    for prop in schema["properties"].values():
        if prop.get("type") in ["object", "array"]:
            return False

    return True

In [ ]:
def preprocess_json_schema(schema: dict) -> Dict[str, str]:
    """
    Preprocess a flat JSON schema to create a mapping of field names to their types.

    Args:
    user_schema (dict): A flat JSON schema.

    Returns:
    Dict[str, str]: A dictionary mapping field names to their types (number, string, bool, or integer).
    """
    assert is_flat_schema(schema), f"JSON schema is not flat: {schema}"
    result = {}
    properties = schema.get("properties", {})

    for field_name, field_info in properties.items():
        field_type = field_info.get("type", "")
        if field_type == "number":
            result[field_name] = "number"
        elif field_type == "string":
            result[field_name] = "string"
        elif field_type == "boolean":
            result[field_name] = "bool"
        elif field_type == "integer":
            result[field_name] = "integer"

    return result

In [ ]:
def jsonschema_type_to_python_type(field_type: str) -> str:
    if field_type == "number":
        return "float"
    elif field_type == "string":
        return "str"
    elif field_type == "boolean":
        return "bool"
    elif field_type == "integer":
        return "int"
    else:
        raise ValueError(f"Unsupported field type: {field_type}")

In [ ]:
def function_config_to_dspy_signature(function_name: str, function_config: dict):
    assert (
        "system_schema" not in function_config
    ), "System schema not supported by DSPy recipe"
    assert (
        "assistants_schema" not in function_config
    ), "Assistant schema not supported by DSPy recipe"
    assert "user_schema" in function_config, "User schema not found in function config"
    user_schema_path = config_path.parent / function_config["user_schema"]
    with user_schema_path.open("r") as f:
        user_schema = preprocess_json_schema(json.load(f))
    output_schema_path = function_config.get("output_schema", None)
    if output_schema_path:
        output_schema_path = config_path.parent / output_schema_path
        with output_schema_path.open("r") as f:
            output_schema = preprocess_json_schema(json.load(f))
    else:
        output_schema = None
    input_signature = ""
    for field_name, field_type in user_schema.items():
        input_signature += f"{field_name}:{jsonschema_type_to_python_type(field_type)},"
    input_signature = input_signature[:-1]
    ## we don't need to trim the trailing comma because DSPy will handle it correctly (ignore it)
    if output_schema:
        output_signature = ""
        for field_name, field_type in output_schema.items():
            output_signature += (
                f"{field_name}:{jsonschema_type_to_python_type(field_type)},"
            )
        # we don't need to trim the trailing comma because DSPy will handle it correctly (ignore it)
    else:
        output_signature = "output"
    string_signature = f"{input_signature} -> {output_signature}"
    return dspy.make_signature(string_signature, signature_name=function_name)

In [ ]:
function_signature = function_config_to_dspy_signature(FUNCTION_NAME, function_config)

Retrieve the database name.


In [ ]:
if "clickhouse" in config and "database" in config["clickhouse"]:
    database_name = config["clickhouse"]["database"]
else:
    database_name = "tensorzero"

Initialize the ClickHouse client.


In [ ]:
assert (
    "CLICKHOUSE_NATIVE_URL" in os.environ
), "CLICKHOUSE_NATIVE_URL environment variable not set"

url_with_database = urljoin(os.environ["CLICKHOUSE_NATIVE_URL"], database_name)

clickhouse_client = Client.from_url(url_with_database)

Retrieve the metric configuration.


In [ ]:
assert "metrics" in config, "No `[metrics]` section found in config"
assert (
    METRIC_NAME in config["metrics"]
), f"No metric named `{METRIC_NAME}` found in config"

metric = config["metrics"][METRIC_NAME]

metric

Determine the ClickHouse table name for the metric.


In [ ]:
feedback_table_name = {
    "float": "FloatMetricFeedback",
    "boolean": "BooleanMetricFeedback",
}.get(metric["type"])

if feedback_table_name is None:
    raise ValueError(f"Unsupported metric type: {metric['type']}")

Grab the dataset of example which were successful according to the metric.


In [ ]:
assert "type" in metric, "Metric is missing the `type` field"
assert "optimize" in metric, "Metric is missing the `optimize` field"

threshold = FLOAT_METRIC_THRESHOLD if metric["type"] == "float" else 0.5
comparison_operator = ">=" if metric["optimize"] == "max" else "<="

query = f"""
SELECT 
    i.variant_name, 
    i.input, 
    i.output, 
    i.episode_id,
    f.value
FROM 
    Inference i
JOIN 
    BooleanMetricFeedback f ON i.id = f.target_id
WHERE 
    i.function_name = %(function_name)s
    AND f.value {comparison_operator} %(threshold)s
LIMIT %(max_samples)s
"""

params = {
    "database_name": database_name,
    "feedback_table_name": feedback_table_name,
    "function_name": FUNCTION_NAME,
    "comparison_operator": comparison_operator,
    "threshold": threshold,
    "max_samples": MAX_SAMPLES,
}

df = clickhouse_client.query_dataframe(query, params)

df.head()

In [ ]:
def parse_dspy_compatible_inputs(input_raw: str) -> Optional[Dict[str, str]]:
    """
    Checks that the input of this Inference is in the correct format for DSPy.
    Then returns the dictionary of inputs.
    """
    try:
        parsed_input = json.loads(input_raw)
    except json.JSONDecodeError:
        logger.warning(f"Input is not valid JSON: {input_raw}")
        return None
    messages = parsed_input.get("messages", None)
    if messages is None:
        logger.warning(f"Input contains no messages: {input_raw}")
        return None
    if len(messages) != 1:
        logger.warning(f"Input contains more than one message: {input_raw}")
        return None
    message = messages[0]
    content = message.get("content", None)
    if content is None:
        logger.warning(f"Input contains no content: {input_raw}")
        return None
    if len(content) != 1:
        logger.warning(f"Input must contain exactly one content item: {input_raw}")
        return None
    content = content[0]
    if content["type"] != "text":
        logger.warning(f"Input contains non-text content: {input_raw}")
        return None
    value = content.get("value", None)
    if value is None:
        logger.warning(f"Input contains no value: {input_raw}")
        return None
    return value

In [ ]:
# Parse the input column into a list of dicts and create a new DataFrame with parsed content
parsed_inputs = df["input"].apply(parse_dspy_compatible_inputs)

# Filter out None values and create a list of dictionaries
valid_inputs = [input_dict for input_dict in parsed_inputs if input_dict is not None]

if valid_inputs:
    # Create a new DataFrame from the list of dictionaries
    parsed_df = pd.DataFrame(valid_inputs)

    # Check for duplicate column names between df and parsed_df
    df_columns = set(df.columns)
    parsed_df_columns = set(parsed_df.columns)
    duplicate_columns = df_columns.intersection(parsed_df_columns)
    assert (
        len(duplicate_columns) == 0
    ), f"Duplicate columns found in parsed_df: {duplicate_columns}"

    # Combine the original DataFrame with the new parsed DataFrame
    result_df = pd.concat([df.reset_index(drop=True), parsed_df], axis=1)
else:
    print("No valid inputs were found after parsing.")

In [ ]:
def parse_dspy_compatible_outputs_chat(output_raw: str) -> Optional[str]:
    try:
        parsed_output = json.loads(output_raw)
    except json.JSONDecodeError:
        logger.warning(f"Output is not valid JSON: {output_raw}")
        return None
    if len(parsed_output) != 1:
        logger.warning(f"Output contains more than one message: {output_raw}")
        return None
    message = parsed_output[0]
    if message["type"] != "text":
        logger.warning(f"Output contains non-text content: {output_raw}")
        return None
    value = message.get("text", None)
    if value is None:
        logger.warning(f"Output contains no value: {output_raw}")
        return None
    return value

In [ ]:
def parse_dspy_compatible_outputs_json(output_raw: str) -> Optional[Dict[str, str]]:
    try:
        parsed_output = json.loads(output_raw)
    except json.JSONDecodeError:
        logger.warning(f"Output is not valid JSON: {output_raw}")
        return None
    parsed_output = parsed_output.get("parsed", None)
    if parsed_output is None:
        logger.warning(f"Output contains no parsed content: {output_raw}")
        return None
    return parsed_output

In [ ]:
def parse_dspy_compatible_outputs(
    output_raw: str, function_config: dict
) -> Optional[Dict[str, str]]:
    if function_config["type"] == "chat":
        chat_output = parse_dspy_compatible_outputs_chat(output_raw)
        if chat_output is None:
            return None
        return {"output": chat_output}
    elif function_config["type"] == "json":
        return parse_dspy_compatible_outputs_json(output_raw)
    else:
        logger.warning(f"Unknown function type: {function_config['type']}")
        return None


parse_outputs = partial(parse_dspy_compatible_outputs, function_config=function_config)

In [ ]:
# Parse the output column and create a new DataFrame with parsed content
parsed_outputs = df["output"].apply(parse_outputs)

# Filter out None values and create a list of dictionaries
valid_outputs = [
    output_dict for output_dict in parsed_outputs if output_dict is not None
]

if valid_outputs:
    # Create a new DataFrame from the list of dictionaries
    parsed_output_df = pd.DataFrame(valid_outputs)

    # Combine the result_df (which already has parsed inputs) with the new parsed outputs
    # Overwrite columns in result_df with the same names from parsed_output_df
    result_df.update(parsed_output_df)
    result_df = pd.concat(
        [
            result_df,
            parsed_output_df[
                [
                    col
                    for col in parsed_output_df.columns
                    if col not in result_df.columns
                ]
            ],
        ],
        axis=1,
    )
    print("Resulting DataFrame:")
    print(result_df.head())
else:
    print("No valid inputs were found after parsing.")

In [ ]:
class TensorZeroDSPyDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        dev_fraction: float = DEV_FRACTION,
        test_fraction: float = TEST_FRACTION,
    ):
        # Ensure unique episode_ids in each set
        unique_episodes = df["episode_id"].unique()
        np.random.shuffle(unique_episodes)

        # Calculate the number of episodes for each set
        total_episodes = len(unique_episodes)
        dev_size = int(total_episodes * dev_fraction)
        test_size = int(total_episodes * test_fraction)
        train_size = total_episodes - dev_size - test_size

        # Split episode_ids
        train_episodes = unique_episodes[:train_size]
        dev_episodes = unique_episodes[train_size : train_size + dev_size]
        test_episodes = unique_episodes[train_size + dev_size :]

        # Create masks for each set
        train_mask = df["episode_id"].isin(train_episodes)
        dev_mask = df["episode_id"].isin(dev_episodes)
        test_mask = df["episode_id"].isin(test_episodes)

        # Split the DataFrame
        self._train = df[train_mask].to_dict(orient="records")
        self._dev = df[dev_mask].to_dict(orient="records")
        self._test = df[test_mask].to_dict(orient="records")
        self.train_size = len(self._train)
        self.dev_size = len(self._dev)
        self.test_size = len(self._test)
        super().__init__(
            train_seed=0,
            train_size=self.train_size,
            eval_seed=0,
            dev_size=self.dev_size,
            test_size=self.test_size,
        )

        print(f"Train set: {len(self._train)} samples")
        print(f"Dev set: {len(self._dev)} samples")
        print(f"Test set: {len(self._test)} samples")

In [ ]:
dataset = TensorZeroDSPyDataset(result_df)

In [ ]:
dspy_function = dspy.Predict(function_signature)


class Predictor(dspy.Module):
    def __init__(self, signature: dspy.Signature):
        super().__init__()
        self.prog = dspy.Predict(signature)

    def forward(self, **inputs):
        return self.prog(**inputs)

You can swap the teleprompter with any of the teleprompting classes supported by DSPy [here](https://dspy-docs.vercel.app/docs/building-blocks/optimizers).


In [ ]:
from dspy.teleprompt import LabeledFewShot

teleprompter = LabeledFewShot(k=5)
optimized_function = teleprompter.compile(
    Predictor(function_signature), trainset=dataset.train
)

We run an example inference to get the prompt from the history.


In [ ]:
optimized_function(**parsed_inputs[0])

Let's parse out the prompt from the history.


In [ ]:
prompt = lm_client.history[-1]["prompt"]
prompt_no_args = "\n\n".join(prompt.split("\n\n")[:-1]) + "\n\n"
for element, element_info in function_signature.model_json_schema()[
    "properties"
].items():
    if element_info["__dspy_field_type"] == "input":
        # Should render to "topic: {{ topic }}", for example (i.e. a minijinja template)
        prompt_no_args += f"{element_info['prefix']} {{{{{element}}}}}"
user_prompt = prompt_no_args

Write the optimized user prompt to a minijinja file


In [ ]:
with open("optimized_user_prompt.minijinja", "w") as f:
    f.write(user_prompt)

We have now produced a DSPy user prompt printed above. You can now add the minijnja file to your config tree and use it in a TensorZero variant.
